Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle
import psutil

Carga de datos

In [71]:
clientes = pd.read_csv('cliente_tabla.csv')
productos = pd.read_csv('producto_tabla.csv')
#sample_submission = pd.read_csv('sample_submission.csv')
town_state  = pd.read_csv('town_state.csv')

In [3]:
train = pd.read_csv('train.csv')

Para hacer seguimiento a la memoria

In [4]:
def variables_por_tamaño():
    variables = {k: sys.getsizeof(v) for k, v in globals().items() if not k.startswith('_')}
    
    variables_en_gb = {k: v / 1024**3 for k, v in variables.items()}
    
    tipos_variables = {k: type(v).__name__ for k, v in globals().items() if not k.startswith('_')}
    
    df = pd.DataFrame({'Variable': list(variables_en_gb.keys()),
                       'Tipo': list(tipos_variables.values()),
                       'Tamaño_GB': list(variables_en_gb.values())})
    
    df = df.sort_values('Tamaño_GB', ascending=False).reset_index(drop=True)
    
    # Mostrar la memoria disponible
    memoria_disponible()
    
    return df



def Gb_por_variable(variable):
    print(sys.getsizeof(variable)/(1024**3))
    memoria_disponible()

def memoria_disponible():
    mem = psutil.virtual_memory()
    mem_total_gb = mem.total / (1024 ** 3)  # Memoria total en GB
    mem_disponible_gb = mem.available / (1024 ** 3)  # Memoria disponible en GB

    print("Memoria Total:", mem_total_gb, "GB")
    print("Memoria Disponible:", mem_disponible_gb, "GB")


In [5]:
df_ventas_pivot = pd.pivot_table(data=train,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Venta_uni_hoy').fillna(0)

nombre_archivo = 'Pkl/df_ventas_pivot.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(df_ventas_pivot, archivo_pkl)

In [6]:
df_devoluciones_pivot = pd.pivot_table(data=train,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Dev_uni_proxima').fillna(0) 

nombre_archivo = 'Pkl/df_devoluciones_pivot.pkl'
with open(nombre_archivo, 'wb') as archivo_pkl:
    pickle.dump(df_devoluciones_pivot, archivo_pkl)

In [14]:
del train

Para Modelar Unds Vendidas

In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def rf_pred(y_true,periodos):
    y_series = pd.Series(y_true)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    forecaster_rf = ForecasterAutoreg(regressor=model,lags=periodos)
    forecaster_rf.fit(y=y_series)
    y_pred = forecaster_rf.predict(steps=periodos)
    return np.array(y_pred)

def rl_pred(y_true, periodos):
    y_series = np.array(y_true)
    model = LinearRegression()
    periodos_pasados = np.arange(1, len(y_true)+1).reshape(-1, 1)
    model.fit(periodos_pasados, y_series)
    periodos_futuros = np.arange(len(y_true) + 1, len(y_true) + periodos + 1).reshape(-1, 1)
    y_pred = model.predict(periodos_futuros)
    return y_pred

def calcular_MSE(y_true,y_pred):
    return np.mean((y_true - y_pred)**2)

def plot(y_true,y_pred,aplica):
    
    if aplica =='S':
        print(y_true)
        plt.plot(np.concatenate((y_true, y_pred)))
        plt.show()
    
def revisar_linealidad(df,id,periodos,graph='N'):
    y_true = df.iloc[id].to_numpy()
    
    y_train = y_true[:y_true.shape[0]-1]
    y_test = y_true[y_true.shape[0]-1]
    
    cof_coerr = np.abs(np.corrcoef(y_true,df_ventas_pivot.columns)[0][1]) #Correlación positiva
    id_name = df_ventas_pivot.index[id]


    if cof_coerr > 0.75: #Solo regresion Lineal
        pred= rl_pred(y_train,periodos)
    if cof_coerr > 0.5: #Combinación regresion Lineal & RandomForest
        pred= ((rl_pred(y_train,periodos) * cof_coerr) + (rf_pred(y_train,periodos) * cof_coerr))
    else: #Solo RandomForest
        pred= rf_pred(y_train,periodos)

    pred = np.clip(pred, a_min=0, a_max=None)

    plot(y_true,pred,aplica=graph)    
    lista_preds = list(pred.astype('int'))
    lista_preds.append(cof_coerr)
    lista_preds.append(calcular_MSE(y_test,pred[0]))
    return id_name,lista_preds

revisar_linealidad(df_ventas_pivot,8,3,graph='N')

((1110, 7, 3301, 15766, 1250),
 [10, 10, 11, 0.5439433649162874, 0.0081269177815803])

In [ ]:
lista_proyecciones = []
for num,codigo in enumerate(df_ventas_pivot.index):
    print(num/df_ventas_pivot.shape[0])
    lista_proyecciones.append(revisar_linealidad(df_ventas_pivot,num,3))

Base de datos test

In [8]:
test  = pd.read_csv('test.csv')
test_pivot = pd.pivot_table(data=test,
                                 index=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'],
                                 columns='Semana',
                                 values='Semana')

Indices faltantes total

In [9]:
indices_train = df_ventas_pivot.index
indices_test = test_pivot.index

indices_faltantes_total = indices_test.difference(indices_train)
indices_faltantes_total.shape

(1355454,)

In [10]:
Diccionario_faltantes={}
def verificar_agencias(col):
    indices_izq = test_pivot.reset_index()[col].unique()
    indices_der = df_ventas_pivot.reset_index()[col].unique()

    faltantes= np.setdiff1d(indices_izq, indices_der)

    Diccionario_faltantes[col]=faltantes
    print(col,str(faltantes))


for col in ['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID']:
    verificar_agencias(col)

#np.setdiff1d(Diccionario_faltantes['Producto_ID'],productos['Producto_ID'].unique())

Agencia_ID []
Canal_ID []
Ruta_SAK [1823 3555 3556 3557 3558 3801 3803 4559 6148 6162 7433 7775 8044 8189
 9349 9562 9585]
Cliente_ID [   2683    5380    5707 ... 9746104 9747048 9780515]
Producto_ID [   98 31203 31211 31655 32026 32224 32421 32591 32798 32820 33053 35191
 35246 36524 36673 37202 37362 37404 37405 37494 37495 37496 37610 37617
 37618 37620 37626 37688 37702 37745 42323 46064 46131 48217]


In [322]:
columnas_per_level={'Agencia_ID':0,'Canal_ID':1,'Ruta_SAK':2,'Cliente_ID':3,'Producto_ID':4}

def buscar_un_registro(df,indices_lista,col,prod_id):
    indx = df.xs(prod_id,  level=columnas_per_level[col], drop_level=False).index    
    lista_ixs=[]
    lista_ixs_solo=[]
    for ix in indx:
        lista_ixs.append({indices_lista.get_loc(ix):ix})
        lista_ixs_solo.append(indices_lista.get_loc(ix))
    return lista_ixs,lista_ixs_solo

buscar_un_registro(test_pivot,indices_test,'Cliente_ID',2683)[1][:5]
#buscar_un_registro(df_ventas_pivot,indices_train,'Producto_ID',36332)[1][:5]

[4190511]

Nombres similares

Ajuste de nombres de productos

In [ ]:
archivo= 'Pkl/Cluster_info.pkl'
# Cargar el DataFrame desde el archivo .pkl
with open(archivo, 'rb') as archivo_pkl:
    Cluster_info = pickle.load(archivo_pkl)

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity


def limpiar_texto(texto):
    texto_limpio = re.sub(r'\b\w{1,2}\b|\d+', '', texto.lower())
    return texto_limpio

productos['NombreProductoLimpio'] = productos['NombreProducto'].apply(limpiar_texto)
productos['NombreProductoLimpio'] = productos['NombreProductoLimpio'].apply(limpiar_texto)
productos = productos.merge(Cluster_info[0],how='left',on='Producto_ID').fillna(9999)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(productos['NombreProductoLimpio'])
count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(),index=productos['Producto_ID'])
similaridades = cosine_similarity(count_df)

Buscar ids

In [ ]:
def Productos_similares(id_a_buscar,graph='N'):
    similares = similaridades[productos[productos['Producto_ID'] == id_a_buscar].index]
    similares_df = pd.DataFrame(similares.T, index=productos['Producto_ID'], columns=['Similitud'])
    productos_similares = similares_df[similares_df.index != id_a_buscar].sort_values(by='Similitud', ascending=False).head(20)
    
    productos_filtrados = productos[productos['Producto_ID'].isin(productos_similares.head(20).index)]
    productos_filtrados = productos_filtrados[productos_filtrados['Cluster']<100]

    if graph=='S':
        display(productos[productos['Producto_ID']==id_a_buscar])
        display(productos[productos['Producto_ID'].isin(productos_similares.index)])
        display(productos_filtrados)

    return productos_filtrados['Producto_ID'].to_numpy()


Productos_similares(31655,graph='S')

In [307]:
def Crear_Matriz_Para_Similaridad(id_clave):
    tupla = test_pivot.iloc[id_clave].name
    Dict_nombres = {}
    Dict_nombres['Producto_ID']=tupla[4]

    ids_productos = Productos_similares(Dict_nombres['Producto_ID'],graph='N')

    #Cluster preferido
    Cluster_moda = Cluster_info[0].loc[ids_productos].groupby('Cluster')['Cluster'].count().sort_values(ascending=False).index[0]
    #Todos los productos del cluster preferido
    Indices_del_cluster_moda = Cluster_info[0][Cluster_info[0]['Cluster']==Cluster_moda].index
    
    
    df_filtrado =  df_ventas_pivot.reset_index()[df_ventas_pivot.reset_index()['Producto_ID'].isin(Indices_del_cluster_moda)][['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID']]
    
    nueva_fila = pd.DataFrame({
        'Agencia_ID': tupla[0],
        'Canal_ID': tupla[1],
        'Ruta_SAK': tupla[2],
        'Cliente_ID': tupla[3],
        'Producto_ID': tupla[4]
    },index=[9999999999999])
    
    df_filtrado = pd.concat([df_filtrado,nueva_fila],axis=0)
    df_filtrado['Clave'] = df_filtrado.apply(lambda row: f"{row['Agencia_ID']} {row['Canal_ID']} {row['Ruta_SAK']} {row['Cliente_ID']} {row['Producto_ID']}", axis=1)
    df_filtrado = df_filtrado.drop(columns=['Agencia_ID','Canal_ID','Ruta_SAK','Cliente_ID','Producto_ID'])


    return df_filtrado#.set_index(indices_claves)


def Similaridad_entre_registros(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['Clave'])
    count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(),index=df.index)
    similaridades = cosine_similarity(count_df)
    df_Similiares = pd.DataFrame(similaridades_2[similaridades_2.shape[0]-1].T,index=df.index,columns=['Similitud'])
    df_Similiares = df_Similiares.sort_values(by='Similitud', ascending=False).head(20)

    return df_Similiares

Similaridad_entre_registros(Crear_Matriz_Para_Similaridad(2807162))



,Similitud
9999999999999,1.00
11442737,0.25
11442738,0.25
25391077,0.25
17039631,0.00
17039659,0.00
17039662,0.00
17039664,0.00
17039688,0.00
17039713,0.00


In [321]:
import warnings
warnings.filterwarnings("ignore")


def Predict_test(df,num):
    
    Dict_nombres = {}
    index_de_test = df.iloc[num].name

    Dict_nombres['Agencia_ID']=index_de_test[0]
    Dict_nombres['Canal_ID']=index_de_test[1]
    Dict_nombres['Ruta_SAK']=index_de_test[2]
    Dict_nombres['Cliente_ID']=index_de_test[3]
    Dict_nombres['Producto_ID']=index_de_test[4]

    if index_de_test in indices_faltantes_total:
        #Verificar que el producto SI EXISTA
        if Dict_nombres['Producto_ID'] not in Diccionario_faltantes['Producto_ID']:
            1
        #Si no existe la tupla verificamos si existe el id producto en train
        else:
            
            #Si no existe el id producto buscamos el similar por distancia de coseno
            ids_similares = Productos_similares(Dict_nombres['Producto_ID'],graph='N')
            #Revisamos cada uno de los resultados
            lista_resultados_similares=[]
            
            for ids_sim in ids_similares:
                #Creamos la nueva tupla con los ids similares
                tuple_id_sim = (Dict_nombres['Agencia_ID'],Dict_nombres['Canal_ID'],Dict_nombres['Ruta_SAK'],Dict_nombres['Cliente_ID'],ids_sim)
                
                if tuple_id_sim in indices_train:
                    lista_resultados_similares.append(revisar_linealidad(df_ventas_pivot,indices_train.get_loc(tuple_id_sim),3,graph='N')[1][1:3])
                
            #Si encontró valores usaremos esos
            if len(lista_resultados_similares)>0:
                print(lista_resultados_similares)
                return np.round(np.array([lista_resultados_similares]).mean(axis=1))
                

            #Si no, vamos a generalizar con los productos que hay
            else:
                
                for ids_sim in ids_similares:
                    id_base = buscar_un_registro(df_ventas_pivot,indices_train,'Producto_ID',ids_sim)[1][0]
                    lista_resultados_similares.append(revisar_linealidad(df_ventas_pivot,id_base,3,graph='N')[1][1:3])
                print(lista_resultados_similares)
                return  np.round(np.array([lista_resultados_similares]).mean(axis=1))
                #Promediar los resultados obtenidos de los productos más similares
                    
    else:
        return revisar_linealidad(df_ventas_pivot,indices_train.get_loc(index_de_test),3,graph='N')[1][1:3]
    
Predict_test(test_pivot,2807162)#6113671 [2806970, 2807091, 2807162, 2807273, 2807636]


[[190, 199], [0, 0], [78, 78], [0, 0], [0, 2], [0, 0], [6, 4], [55, 53], [0, 0], [0, 0], [0, 0]]


array([[30., 31.]])

In [721]:
(22560, 1, 1077, 801976, 1876) in indices_faltantes_total

False

In [719]:
indices_faltantes_total

MultiIndex([( 1110, 7, 3301,   50395,   713),
            ( 1110, 7, 3301,   73589,  1644),
            ( 1110, 7, 3301,  181232, 47336),
            ( 1110, 7, 3301,  198780, 42869),
            ( 1110, 7, 3301,  319641,   325),
            ( 1110, 7, 3301,  319641,  1212),
            ( 1110, 7, 3301,  319641,  1643),
            ( 1110, 7, 3301,  319641, 31688),
            ( 1110, 7, 3301,  319641, 32936),
            ( 1110, 7, 3301,  319641, 32961),
            ...
            (25759, 1, 5517, 4483380, 30694),
            (25759, 1, 5517, 4483380, 31005),
            (25759, 1, 5517, 4483380, 34255),
            (25759, 1, 5517, 4483380, 37024),
            (25759, 1, 5517, 4528866, 35142),
            (25759, 1, 5517, 4565941, 36164),
            (25759, 1, 5517, 4565941, 37581),
            (25759, 1, 5517, 4707699, 34226),
            (25759, 1, 5517, 4707699, 36598),
            (25759, 1, 5517, 4744611, 34255)],
           names=['Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Client